In [7]:
import os
import yaml
import shutil
from pathlib import Path
import random
import soundfile as sf
import numpy as np

In [8]:

def organize_tracks(base_dir):
    track_dirs = [d for d in os.listdir(base_dir) if d.startswith("Track") and os.path.isdir(os.path.join(base_dir, d))]

    for track_dir in track_dirs:
        metadata_path = os.path.join(base_dir, track_dir, "metadata.yaml")
        stems_dir = os.path.join(base_dir, track_dir, "stems")

        if not os.path.exists(metadata_path) or not os.path.exists(stems_dir):
            continue

        with open(metadata_path, 'r') as f:
            metadata = yaml.safe_load(f)

        for stem_key, stem_data in metadata.get('stems', {}).items():
            inst_class = stem_data.get('inst_class', "Unknown")

            inst_folder = os.path.join(base_dir, inst_class)
            os.makedirs(inst_folder, exist_ok=True)

            stem_file_name = f"{stem_key}.wav"
            stem_file_path = os.path.join(stems_dir, stem_file_name)

            if os.path.exists(stem_file_path):
                target_file_name = f"{track_dir}_{stem_file_name}"
                target_file_path = os.path.join(inst_folder, target_file_name)

                shutil.copy2(stem_file_path, target_file_path)


In [6]:
folder = '/Users/francesco/Desktop/babyslakh_16k'
organize_tracks(folder)

In [9]:
def create_mixture_and_save(base_dir):
    track_dirs = [d for d in os.listdir(base_dir) if d.startswith("Track") and os.path.isdir(os.path.join(base_dir, d))]

    for track_dir in track_dirs:
        metadata_path = os.path.join(base_dir, track_dir, "metadata.yaml")
        stems_dir = os.path.join(base_dir, track_dir, "stems")

        if not os.path.exists(metadata_path) or not os.path.exists(stems_dir):
            continue

        with open(metadata_path, 'r') as f:
            metadata = yaml.safe_load(f)

        mixture = None
        sample_rate = None

        for stem_key, stem_data in metadata.get('stems', {}).items():
            inst_class = stem_data.get('inst_class', "Unknown")
            if inst_class in ['Bass', 'Vocals', 'Drums']:
                continue

            stem_file_name = f"{stem_key}.wav"
            stem_file_path = os.path.join(stems_dir, stem_file_name)

            if os.path.exists(stem_file_path):
                data, sr = sf.read(stem_file_path)
                if mixture is None:
                    mixture = data
                    sample_rate = sr
                else:
                    mixture = mixture[:len(data)] + data[:len(mixture)]

        if mixture is not None:
            max_val = np.max(np.abs(mixture))
            if max_val > 0:
                mixture = mixture / max_val

            total_samples = len(mixture)
            six_sec_samples = sample_rate * 6
            if total_samples > six_sec_samples:
                start_idx = random.randint(0, total_samples - six_sec_samples)
                mixture = mixture[start_idx:start_idx + six_sec_samples]
                
            output_path = os.path.join(base_dir, track_dir, "audio.wav")
            sf.write(output_path, mixture, sample_rate)

In [10]:
create_mixture_and_save(folder)